## Ejercicio 1


### Importamos las librerias necesarias


In [351]:
import numpy as np
import pandas as pd


<p>Todas las hojas de calculo se encuentran en la carpeta <em>data</em>, desde ahi vamos a importar todos los archivos que necesitamos.</p>

<p>Como primer paso es necesario leer los archivos a transformar en este caso son <em>principal.xlsx</em> y <em>secundario.xlsx</em></p>


In [352]:
df_principal = pd.read_excel(r"data/principal.xlsx")
df_secundario = pd.read_excel(r"data/secundario.xlsx")
df_tipo_cambio = pd.read_excel(r"data/tipocambio.xlsx")


## Archivo Principal.xlsx


### Eliminar columnas Codigo y Pases de principal.xlsx


In [353]:
df_principal.drop(columns=["Codigo", "Pases"], axis=1, inplace=True)
# print(df_principal.head())


### Dejar unicamente el nombre de la pelicula en la columna NombreP


In [354]:
"""
Los datos de la columna NombreP tienen el formato:
    Nombre (Caractetisticas)
para obtener el nombre se puede utilizar la funcion split para separar el string
el primer valor del array es el que nos interesa
"""


def get_movie_name(row):
    return row.split("(")[0]


### Agupar las peliculas con la sumatoria de EntradasTotales, Dinero y Funciones


In [355]:
# Guardamos en una lista la columna NombreP
lista_nombres = df_principal["NombreP"].tolist()

# A la columna NombreP le aplicamos la funcion get_movie_name para solo tener el nombre de la pelicula
df_principal["NombreP"] = df_principal["NombreP"].map(get_movie_name)


# Nuevo dataframe con los nuevos datos
df_principal_nuevo = pd.DataFrame()

# copia las columnas necesarias
df_principal_nuevo["Teatro"] = df_principal["Teatro"]
df_principal_nuevo[""] = df_principal["NombreP"]
df_principal_nuevo["Sala"] = df_principal["Sala"]
df_principal_nuevo["Dia"] = df_principal["Dia"]

# Elimina elementos duplicados
df_principal_nuevo.drop_duplicates(
    subset=["Teatro", "", "Sala", "Dia"], keep="first", inplace=True
)

"""
Las columnas Entradas, Dinero y Funciones es la sumatoria de todas las funciones que cumplen estas caracteristicas:
1.- El dia es el mismo
2.- El teatro es el mismo
3.- La sala es la misma
4.- NombreP tiene los mismos atributos 
"""

df_principal_nuevo["Entradas"] = df_principal.groupby(
    ["Teatro", "NombreP", "Sala", "Dia"]
)["EntradasTotales"].transform("sum")
df_principal_nuevo["Dinero"] = df_principal.groupby(
    ["Teatro", "NombreP", "Sala", "Dia"]
)["Dinero"].transform("sum")
df_principal_nuevo["Funciones"] = df_principal.groupby(
    ["Teatro", "NombreP", "Sala", "Dia"]
)["funciones"].transform("sum")
df_principal_nuevo.to_csv(r"data/principal_clean.csv", index=False)


## Archivo Secundario.xlsx


### Quitar las primeras 4 filas y la primera columna


In [356]:
df_secundario.drop(df_secundario.index[range(4)], axis=0, inplace=True)
df_secundario.drop(df_secundario.columns[[0]], axis=1, inplace=True)

# Renombramos las columnas
df_secundario.columns = [
    "Date",
    "Cinema",
    "Screen",
    "Film",
    "Admits",
    "Cin Box Office Gross",
]


### Transforma las fechas con el mismo formato que en el archivo principal.xlsx


In [357]:
"""
En el archivo principal.xlsx las fechas tienen el formato dd/mm/aaaa
para convertirlas a un formato de fecha se puede utilizar la funcion strptime
"""

# Primero convertimos la columan Date a una columna de tipo datetime


def convert_date(cell):
    if cell == "Total" or pd.isnull(cell):
        return cell
    new_date = pd.to_datetime(cell, errors="coerce")
    return new_date.strftime("%Y-%m-%d")


# Aplicamos la funcion convert_date a la columna Date
df_secundario["Date"] = df_secundario["Date"].map(convert_date)


### Todas las filas deben tener una fecha, a que sala pertenece la funcion e informacion de screen


In [358]:
df_secundario["Date"] = df_secundario["Date"].ffill()
df_secundario["Cinema"] = df_secundario["Cinema"].ffill()
df_secundario["Screen"] = df_secundario["Screen"].ffill()


### Eliminar filas que contenga totales


In [359]:
"""
Las unicas columnas que contiene totales son Screen y Film
entonces solo se debe eliminar filas que en sus columnas Screen o Film tengan como valor Total
"""

df_secundario.drop(
    df_secundario[df_secundario["Screen"] == "Total"].index, axis=0, inplace=True
)
df_secundario.drop(
    df_secundario[df_secundario["Film"] == "Total"].index, axis=0, inplace=True
)


def split_cinema(cell):
    return cell.split(" ")[1]


# A la columna Cinema solo dejamos el numero
df_secundario["Cinema"] = df_secundario["Cinema"].map(split_cinema)


### Separar los datos de Date, Cinema, Screen y Film y eliminar atributos de la columna Film


In [360]:
# nuevo dataframe para los datos limpios
df_secundario_nuevo = pd.DataFrame()
df_secundario_nuevo["Date"] = df_secundario["Date"]
df_secundario_nuevo[""] = df_secundario["Cinema"]
df_secundario_nuevo["Screen"] = df_secundario["Screen"]

"""
Anteriormente se hizo una funcion para eliminar los atributos de las peliculas
para este caso solo vamos a reutilizar la funcion get_movie_name
"""

df_secundario["Film"] = df_secundario["Film"].map(get_movie_name)
df_secundario_nuevo["Film"] = df_secundario["Film"]


### Eliminar datos duplicados y sumar las entradas totales de las funciones


In [361]:
df_secundario.drop_duplicates(
    subset=["Date", "Cinema", "Screen", "Film", "Admits"], keep="first", inplace=True
)

df_secundario_nuevo.drop_duplicates(
    subset=["Date", "", "Screen", "Film"], keep="first", inplace=True
)

# Agrupamos las columnas Admits, Cin Box Office Gross
df_secundario_nuevo["Admits"] = df_secundario.groupby(
    ["Date", "Cinema", "Screen", "Film"]
)["Admits"].transform("sum")
df_secundario_nuevo["Cin Box Office Gross"] = df_secundario.groupby(
    ["Date", "Cinema", "Screen", "Film"]
)["Cin Box Office Gross"].transform("sum")
df_secundario_nuevo.to_csv(r"data/secundario_clean.csv", index=False)


### Agrupacion de datos


In [362]:
# Cambio de tipo de datos de las columnas

df_principal_nuevo["Teatro"] = pd.to_numeric(
    df_principal_nuevo["Teatro"], downcast="integer"
)
df_principal_nuevo["Sala"] = pd.to_numeric(
    df_principal_nuevo["Sala"], downcast="integer"
)
df_principal_nuevo["Dia"] = pd.to_datetime(df_principal_nuevo["Dia"], errors="coerce")

df_secundario_nuevo["Date"] = pd.to_datetime(
    df_secundario_nuevo["Date"], errors="coerce"
)
df_secundario_nuevo[""] = pd.to_numeric(df_secundario_nuevo[""], downcast="integer")
df_secundario_nuevo["Screen"] = pd.to_numeric(
    df_secundario_nuevo["Screen"], downcast="integer"
)

df_union = pd.merge(
    df_principal_nuevo,
    df_secundario_nuevo,
    left_on=["Teatro", "", "Sala", "Dia"],
    right_on=["", "Film", "Screen", "Date"],
    how="left",
    copy=False,
)

df_union.rename(columns={"_x": ""}, inplace=True)

# elimina columnas no necesarias que se crearon en el merge
df_union.drop(["_y", "Screen", "Film", "Date"], axis=1, inplace=True)
df_union.to_csv(r"data/union.csv", index=False)


### Restas


In [363]:
# Primero creamos el dataframe final
df_final = pd.DataFrame()

# copiamos las columnas Teatro, "", Sala, Dia de df_union
df_final["Teatro"] = df_union["Teatro"]
df_final[""] = df_union[""]
df_final["Sala"] = df_union["Sala"]
df_final["Dia"] = df_union["Dia"]

# Restamos al valor de la columna Entradas la columna Admits
df_final["EntradasRestadas"] = df_union["Entradas"] - df_union["Admits"]

# Copiamos la columna Admits de df_union
df_final["Admits"] = df_union["Admits"]

# Restamos al valor de la columna Dinero la columna Cin Box Office Gross
df_final["DineroRestado"] = df_union["Dinero"] - df_union["Cin Box Office Gross"]

# Copiamos las columnas Cin Box Office Gross y Funciones de df_union
df_final["Cin Box Office Gross"] = df_union["Cin Box Office Gross"]
df_final["Funciones"] = df_union["Funciones"]

df_final.to_csv(r"data/final.csv", index=False)


### Tipo de cambio


In [364]:
# cambia el nombre de la columna Cinema de df_tipo_cambio a Teatro
df_tipo_cambio.rename(columns={"Cinema": "Teatro"}, inplace=True)


df_a_cambiar = pd.DataFrame()
df_a_cambiar = df_final.merge(df_tipo_cambio, on=["Teatro"])

# dividimos df_a_cambiar["DineroRestado"] entre df_a_cambiar["ExgRate"] redondeado a 2 decimales
df_a_cambiar["DineroRestado Cambio"] = (
    df_a_cambiar["DineroRestado"] / df_a_cambiar["ExgRate"]
)
df_a_cambiar["Cin Gross Cambio"] = (
    df_a_cambiar["Cin Box Office Gross"] / df_a_cambiar["ExgRate"]
)

df_a_cambiar["Cin Gross Cambio"] = pd.to_numeric(
    df_a_cambiar["Cin Gross Cambio"], downcast="float"
)

df_a_cambiar["DineroRestado Cambio"] = pd.to_numeric(
    df_a_cambiar["DineroRestado Cambio"], downcast="float"
)

df_a_cambiar["DineroRestado Cambio"] = df_a_cambiar["DineroRestado Cambio"].round(2)
df_a_cambiar["Cin Gross Cambio"] = df_a_cambiar["Cin Gross Cambio"].round(2)

# Cambiamos el formato de df_a_cambiar["Dia"] a dd/mm/yyyy
df_a_cambiar["Dia"] = df_a_cambiar["Dia"].dt.strftime("%d/%m/%Y")

# Quitamos las columnas Dinerorestado y Cin Box Office Gross
df_a_cambiar.drop(
    ["DineroRestado", "Cin Box Office Gross", "ExgRate"], axis=1, inplace=True
)

# Quitamos el nombre de columna
df_a_cambiar.rename(columns={"EntradasRestadas": ""}, inplace=True)
df_a_cambiar.rename(columns={"Admits": ""}, inplace=True)
df_a_cambiar.rename(columns={"DineroRestado Cambio": ""}, inplace=True)
df_a_cambiar.rename(columns={"Cin Gross Cambio": ""}, inplace=True)
df_a_cambiar.rename(columns={"Funciones": ""}, inplace=True)

# Exportamos a excel el dataframe df_a_cambiar
df_a_cambiar.to_excel(r"data/final_ejercicio_1.xlsx", index=False, header=True)
